In [1]:
%load_ext autoreload
%autoreload 2
from EventDriven.riskmanager import RiskManager, LOOKBACKS, close_cache, chain_cache, oi_cache, spot_cache
from dbase.DataAPI.ThetaData import list_contracts
from trade.assets.Calculate import Calculate
from trade.helpers.helper import parse_option_tick, binomial_implied_vol
from trade.assets.rates import get_risk_free_rate_helper
from dateutil.relativedelta import relativedelta
from dbase.DataAPI.ThetaData import retrieve_eod_ohlc
from pandas.tseries.offsets import BDay
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import math


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-04-03 00:04:17 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [5]:
from trade.helpers.openbb_helper import load_openBB
from openbb import obb
data = obb.equity.price.historical(symbol='AAPL', start_date = '2021-01-01', end_date = '2025-04-04', provider='yfinance', interval ='1d').to_df()
data
# load_openBB()

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2021-01-04,133.520004,133.610001,126.760002,129.410004,143301900,0.0,0.0
2021-01-05,128.889999,131.740005,128.429993,131.009995,97664900,0.0,0.0
2021-01-06,127.720001,131.050003,126.379997,126.599998,155088000,0.0,0.0
2021-01-07,128.360001,131.630005,127.860001,130.919998,109578200,0.0,0.0
2021-01-08,132.429993,132.630005,130.229996,132.050003,105158200,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-31,217.009995,225.619995,216.229996,222.130005,65299300,0.0,0.0
2025-04-01,219.809998,223.679993,218.899994,223.190002,36412700,0.0,0.0
2025-04-02,221.320007,225.190002,221.020004,223.889999,35905900,0.0,0.0


In [2]:
from EventDriven.portfolio import OptionSignalPortfolio
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
from functools import partial


# ***Extending Risk Manager for greek handling***

### ***INITIAL BACKTEST RUN***

In [3]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
_key = 10
with open(f'../input/profitable_weights_{_key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{_key}.csv').iloc[:, 1:]
ttrades__['Duration'] = ttrades__.Duration.apply(lambda x: int(x.split(' ')[0]))
# AMZN20220329LONG
tick = ['NFLX', 'META','SBUX']
ttrades__ = ttrades__[(ttrades__.Ticker.isin(tick))]
trades_ = ttrades__.copy()
# trades_.loc[17, 'Size'] = -126
# ttrades__
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,105,504,551,103.049417,100.110001,-308.638768,-0.028524,2023-01-04,2023-03-14,69,SBUX
3,14,517,687,358.781354,382.399994,330.660962,0.065830,2023-01-24,2023-09-27,246,NFLX
4,26,521,752,149.932939,358.989990,5435.483341,1.394337,2023-01-30,2023-12-29,333,META
8,96,575,587,109.251048,104.269997,-478.180907,-0.045593,2023-04-18,2023-05-04,16,SBUX
12,13,704,752,407.049710,490.369995,1083.163708,0.204693,2023-10-20,2023-12-29,70,NFLX
14,96,717,735,104.103088,97.379997,-645.416698,-0.064581,2023-11-08,2023-12-05,27,SBUX


In [4]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)


max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash
pd.Series(weights).sort_values(ascending=False), max_cash

(SBUX    0.130273
 NFLX    0.064065
 META    0.047216
 dtype: float64,
 {'SBUX': 2, 'NFLX': 2, 'META': 2})

In [5]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo

pd.options.display.max_rows = 15
pd.options.display.max_columns = 15

evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)
evb_backtest.portfolio.initial_capital
w_map = {x: w  * 0.85 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 50
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.sizing_lev = 4.5
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .85,
   'dte': 365,
   'moneyness_width': 0.10},
   {'direction': 'short',
  'rel_strike': .80,
  'dte': 365,
  'moneyness_width': 0.10}
],
 'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.portfolio.min_moneyness_threshold = 5
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 90
evb_backtest.portfolio.moneyness_width_factor = .025
evb_backtest.portfolio.dte_reduction_factor = 30
evb_backtest.portfolio.min_acceptable_dte_threshold = 95
evb_backtest.portfolio.risk_manager.limits['dte'] = False
evb_backtest.portfolio.risk_manager.limits['moneyness'] = True
evb_backtest.portfolio.risk_manager.max_moneyness = 1.3
for key  in max_cash:
    if max_cash[key]*100 > evb_backtest.portfolio.allocated_cash_map[key]:
        print(key, max_cash[key]*100, evb_backtest.portfolio.allocated_cash_map[key])

evb_backtest.portfolio.risk_manager.print_settings()

signals = evb_backtest.bars.signal_df
signals_df = deepcopy(signals).set_index('Date')
((signals_df!=-1)&(signals_df!=-0)).sum().sum()
# signals_df

Saving to cache from db

Risk Manager Settings:
Start Date: 2023-01-04 00:00:00
End Date: 2023-12-29 00:00:00
Current Limits State (Position Adjusted when these thresholds are reached):
    Delta: True
    Gamma: False
    Vega: False
    Theta: False
    Roll On DTE: False
        Min DTE Threshold: 95
    Roll On Moneyness: True
        Max Moneyness: 1.3
Quanitity Sizing Type: delta
            


6

In [6]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Signal ID: SBUX20230104LONG
Updating Cache
Order Produced: {'result': 'SUCCESSFUL', 'data': {'long': ['SBUX20240119C130'], 'short': ['SBUX20240119C140'], 'trade_id': '&L:SBUX20240119C130&S:SBUX20240119C140', 'close': 1.9250000000000003}}
Position ID: &L:SBUX20240119C130&S:SBUX20240119C140
Calculating Position Greeks
['L', 'SBUX20240119C130']
['S', 'SBUX20240119C140']
202 Zero values found for SBUX20240119C140. Resolving missing vol
76 Zero values found for SBUX20240119C130. Resolving missing vol
Interpolated Vol Resolved for  SBUX20240119C140  on  2023-01-11
Interpolated Vol Resolved for  SBUX20240119C130  on  2023-01-05
Interpolated Vol Resolved for  SBUX20240119C140  on  2023-01-11
Interpolated Vol Resolved for  SBUX20240119C130  on  2023-01-05
Interpolated Vol Resolved for  SBUX20240119C140  on  2023-01-13
Interpolated Vol Resolved for  SBUX20240119C130  on  2023-02-14
Interpolated Vol Resolved for  SBUX20240119C140  on  2023-01-13
I

In [7]:
evb_backtest.start_date, evb_backtest.end_date

(Timestamp('2023-02-03 00:00:00'), Timestamp('2023-12-29 00:00:00'))

In [79]:
from trade.assets.helpers.DataManagers import OptionDataManager  
dm = OptionDataManager(opttick='AAPL20240119C175')

In [81]:
from trade.assets.helpers.DataManagers import OptionDataManager  
dm = OptionDataManager(opttick='AAPL20240119C175')
data = dm.get_timeseries('2023-02-03', '2023-12-03', '1d', 'greeks')
data

OptionDataManager calculating greeks. Database unavailable


In [22]:
%load_ext autoreload
%autoreload 2
from trade.assets.helpers.DataManagers import OptionDataManager
data_manager = OptionDataManager(opttick='AAPL20240322C170', default_fill='midpoint')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
data = data_manager.get_timeseries(
    start = '2024-01-01',
    end = '2024-12-31',
    interval = '1d',
    type_ = 'spot'
)
data

,Open,High,Low,Close,Midpoint,Volume
Datetime,,,,,,
2024-02-01,19.00,19.00,19.00,19.00,19.175,1
2024-02-02,12.95,18.75,12.95,18.75,18.075,3
2024-02-05,19.55,19.55,19.55,19.55,19.425,2
2024-02-06,19.70,19.70,19.70,19.70,20.850,1
2024-02-07,20.93,21.58,20.25,20.75,20.850,6
...,...,...,...,...,...,...
2024-03-18,5.95,8.00,4.30,4.40,4.425,3658
2024-03-19,4.95,6.83,3.85,6.10,6.375,1776
2024-03-20,6.00,8.75,5.40,8.75,8.675,613


In [ ]:
use_cols = [x for x in data.columns if 'Midpoint' in x]
data2 = data[use_cols]
data2.columns = [x.split('_')[1].capitalize() for x in data2.columns]
data2

In [67]:
# evb_backtest.risk_manager.position_data['&L:AMD20220617P80&S:AMD20220617P75']
# evb_backtest.portfolio.events.advance_date()
evb_backtest.risk_manager.calculate_position_greeks('&L:AAPL20240315C170&S:AAPL20240315C175', '2023-10-20')
# v = evb_backtest.risk_manager.limits_check()

['L', 'AAPL20240315C170']
['S', 'AAPL20240315C175']


In [63]:
print((10.1 + 12.7 + 15.0 + 11.6)/4)
print((6.9 + 9.1 + 8.8 + 9.1 + 9.7)/5)

12.35
8.719999999999999


In [54]:
[[], []]

[[], []]

In [39]:
evb_backtest.risk_manager.position_data['&L:AAPL20240315C170&S:AAPL20240315C175']

,Delta,Gamma,Theta,Rho,Vega,Vanna,Volga,Midpoint,S,R,Y,S0_close,s0_close
Datetime,,,,,,,,,,,,,
2023-03-06,0.043469,-0.000003,-0.001127,0.049475,0.008352,-0.121052,-13.587082,1.875,0.0,0.0,0.0,0.0,153.830002
2023-03-07,0.045053,-0.000054,-0.001517,0.049787,0.011670,-0.113807,-15.268664,1.975,0.0,0.0,0.0,0.0,151.600006
2023-03-08,0.044645,-0.000027,-0.001330,0.050112,0.009961,-0.120422,-14.568089,1.925,0.0,0.0,0.0,0.0,152.869995
2023-03-09,0.041884,0.000116,-0.001147,0.047505,0.012243,-0.095080,-14.476334,1.650,0.0,0.0,0.0,0.0,150.589996
2023-03-10,0.045343,-0.000021,-0.001785,0.048914,0.016040,-0.095283,-16.465113,1.925,0.0,0.0,0.0,0.0,148.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-20,0.065127,-0.000915,-0.001035,0.033691,-0.014997,-0.353637,9.786682,2.925,0.0,0.0,0.0,0.0,172.880005
2023-10-23,0.066135,-0.000966,-0.001087,0.033629,-0.015245,-0.362834,10.242012,2.950,0.0,0.0,0.0,0.0,173.000000
2023-10-24,0.067072,-0.001022,-0.000966,0.034052,-0.016610,-0.383954,12.299860,2.975,0.0,0.0,0.0,0.0,173.440002


In [29]:
from trade.helpers.decorators import log_time
log_time?

Signature: log_time(logger)
Docstring: <no docstring>
File:      ~/cloned_repos/QuantTools/trade/helpers/decorators.py
Type:      function

### ***RISK MANAGER CORE METHOD TESTS***

In [90]:
test_date = pd.to_datetime('2021-07-15')
position_str = '&L:AMD20220617C115&S:AMD20220617C125'
def parse_position_id(positionID):
    position_str = positionID
    position_list = position_str.split('&')
    position_list = [x.split(':') for x in position_list if x]
    position_list_parsed = [(x[0], parse_option_tick(x[1])) for x in position_list]
    position_dict = dict(L = [], S = [])
    for x in position_list_parsed:
        position_dict[x[0]].append(x[1])
    return position_dict, position_list

def get_position_dict(positionID):
    return parse_position_id(positionID)[0]

def get_position_list(positionID):
    return parse_position_id(positionID)[1]

def get_option_price(optID, portfolio, date):
    return portfolio.options_data[optID]['Midpoint'][date]

    


In [329]:
# evb_backtest.risk_manager.calculate_position_greeks(position_str,test_date)
evb_backtest.risk_manager.calculate_quantity(position_str, 'AMD20210701LONG', '2021-07-01')
# evb_backtest.risk_manager.update_signal_limits(fill_event)

Spot Price at Purchase: 93.30999755859375 at time 2021-07-01 00:00:00
Target Delta: 1.45
Delta from Full Spend: 0.9306110124412736, Size: 13
Delta with Size Limit: 1.4317092499096518, Size: 20


13

In [237]:
evb_backtest.portfolio.current_positions

{'AMD': {'position': {'long': ['AMD20220617C115'],
   'short': ['AMD20220617C125'],
   'trade_id': '&L:AMD20220617C115&S:AMD20220617C125',
   'close': 1.825000000000001},
  'quantity': 11,
  'entry_price': 2270.5487409714515,
  'market_value': 2007.500000000001,
  'signal_id': 'AMD20210701LONG'}}

In [239]:
evb_backtest.risk_manager.limits_check()

KeyError: 'delta'

In [203]:
events = evb_backtest.events.events.copy()
events[events.type == 'FILL'].position[0]

{'long': ['AMD20220617C115'],
 'short': ['AMD20220617C125'],
 'trade_id': '&L:AMD20220617C115&S:AMD20220617C125',
 'close': 1.3499999999999996}

In [204]:
from EventDriven.event import FillEvent
fill_event = FillEvent(datetime = '2021-07-01', 
                       symbol ='AMD', 
                       exchange = 'I', 
                       quantity=14, 
                       direction = 'LONG', 
                       fill_cost = 0,
                       signal_id='AMD20210701LONG',
                       commission = 0.13,
                       position = {'long': ['AMD20220617C115'],
                                'short': ['AMD20220617C125'],
                                'trade_id': '&L:AMD20220617C115&S:AMD20220617C125',
                                'close': 1.3499999999999996})
evb_backtest.risk_manager.update_signal_limits(fill_event)

Equivalent Delta Size: 1.0


In [175]:
evb_backtest.portfolio.trades['SignalID']

0    AMD20210701LONG
Name: SignalID, dtype: object

In [193]:
evb_backtest.portfolio.weight_map['AMD'] * 20_000

2729.761012667402

In [194]:
evb_backtest.portfolio.allocated_cash_map['AMD']

1937.393900773914

In [164]:
evb_backtest.risk_manager.position_data['&L:AMD20220617C115&S:AMD20220617C125']

,Delta,Gamma,Vega,Theta,Rho,Vanna,Volga,Midpoint,Iv,S0_close,s0_close
Datetime,,,,,,,,,,,
2021-07-01,0.071585,0.000779,0.025448,-0.001411,0.045023,-0.020942,-12.324034,2.000,-0.002992,0.0,93.309998
2021-07-02,0.075887,0.000655,0.025141,-0.001609,0.047353,-0.035179,-13.377762,2.250,0.001486,0.0,94.699997
2021-07-06,0.081192,0.000547,0.027271,-0.002157,0.049014,-0.040630,-14.317299,2.500,0.009417,0.0,94.470001
2021-07-07,0.071561,0.000969,0.030466,-0.001801,0.044008,0.001789,-11.686844,1.825,-0.001312,0.0,90.540001
2021-07-08,0.076214,0.001015,0.034979,-0.002298,0.046413,0.007265,-12.449755,1.900,0.004621,0.0,89.739998
2021-07-09,0.073918,0.001033,0.032556,-0.001937,0.045801,0.003379,-12.263898,1.850,-0.000228,0.0,90.900002
2021-07-12,0.079302,0.001000,0.035748,-0.002416,0.048288,0.001074,-13.313481,2.000,0.006196,0.0,90.809998
2021-07-13,0.074486,0.001058,0.033682,-0.002101,0.045352,0.007532,-12.066100,1.825,0.001261,0.0,90.260002
2021-07-14,0.069136,0.001261,0.033893,-0.001829,0.042637,0.025277,-10.495140,1.550,-0.004616,0.0,89.050003


In [152]:
cash_available = w_map['AMD'] * cash
purchase_date = pd.to_datetime('2021-07-01')
s0_at_purchase = data['s'][purchase_date]
leverage = 5
equivalent_size = (math.floor(cash_available/s0_at_purchase)/100) * leverage
equivalent_size/0.072

20.13888888888889

In [127]:
evb_backtest.portfolio.allocated_cash_map['AMD']
# t = get_position_dict(position_str)
# key = list(t.keys())[0]
# t[key][0]['ticker']

2022.1066637737406

In [100]:
pd.options.display.max_columns = 100
## Switching to calculating vol for entire timeseries
data = evb_backtest.portfolio.options_data['AMD20220617C115'].copy()
option_meta = parse_option_tick('AMD20220617C115')
data[['symbol', 'put_call', 'exp_date', 'strike']] = pd.Series(option_meta)
data['s'] = evb_backtest.risk_manager.spot_timeseries['AMD']
data['r'] = evb_backtest.risk_manager.rf_timeseries
data['y'] = evb_backtest.risk_manager.dividend_timeseries['AMD']
data['iv'] = data.apply(lambda x: binomial_implied_vol(price = x['Midpoint'],
                        S = x['s'],
                        K = x['strike'],
                        r = x['r'],
                        exp_date = x['exp_date'],
                        option_type = x['put_call'].lower(),
                        pricing_date = test_date,
                        dividend_yield= x['y']), axis=1)

greeks = data.apply(lambda x: Calculate.greeks(S = x['s'],
                                    K = x['strike'],
                                    r = x['r'],
                                    y = x['y'],
                                    start = test_date,
                                    exp = x['exp_date'],
                                    flag = x['put_call'],
                                    sigma = x['iv']), axis=1, result_type = 'expand')
data = data.join(greeks)


,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint,symbol,put_call,exp_date,strike,s,r,y,iv,Delta,Gamma,Vega,Theta,Rho,Vanna,Volga
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-07-01,8.49,8.49,8.00,8.15,55,577,7.90,274,9.00,8.450,8.254172,AMD,C,2022-06-17,115.0,93.309998,0.00040,0,0.433572,0.384864,0.009835,0.342548,-0.022095,0.253605,0.629526,16.516995
2021-07-02,8.05,9.27,8.05,9.24,847,8,8.65,99,9.35,9.000,9.297664,AMD,C,2022-06-17,115.0,94.699997,0.00038,0,0.433861,0.398658,0.009781,0.351101,-0.022661,0.265481,0.618915,14.458817
2021-07-06,9.05,9.25,8.65,8.65,11,233,8.85,149,9.20,9.025,8.986518,AMD,C,2022-06-17,115.0,94.470001,0.00040,0,0.437131,0.398389,0.009729,0.350185,-0.022774,0.264183,0.617368,14.426723
2021-07-07,8.00,8.00,7.26,7.30,15,2,6.40,558,7.65,7.025,7.645536,AMD,C,2022-06-17,115.0,90.540001,0.00043,0,0.423107,0.350404,0.010070,0.322235,-0.020284,0.227589,0.654788,21.880235
2021-07-08,6.40,6.88,6.40,6.88,3,443,6.10,507,7.00,6.550,6.580316,AMD,C,2022-06-17,115.0,89.739998,0.00043,0,0.416510,0.337650,0.010178,0.314970,-0.019517,0.218922,0.663905,23.972143
2021-07-09,0.00,0.00,0.00,0.00,0,90,6.65,352,6.90,6.775,6.849095,AMD,C,2022-06-17,115.0,90.900002,0.00043,0,0.411459,0.345900,0.010265,0.321979,-0.019710,0.227281,0.665506,23.034816
2021-07-12,0.00,0.00,0.00,0.00,0,42,6.60,34,6.75,6.675,6.667105,AMD,C,2022-06-17,115.0,90.809998,0.00043,0,0.409482,0.343566,0.010299,0.320846,-0.019547,0.225912,0.667930,23.464894
2021-07-13,0.00,0.00,0.00,0.00,0,388,5.90,1,7.20,6.550,5.903342,AMD,C,2022-06-17,115.0,90.260002,0.00045,0,0.411236,0.339187,0.010267,0.317351,-0.019417,0.221733,0.667836,24.010453
2021-07-14,6.15,6.15,6.09,6.09,2,462,5.70,44,6.00,5.850,5.726087,AMD,C,2022-06-17,115.0,89.050003,0.00045,0,0.400286,0.318538,0.010424,0.305264,-0.018180,0.207801,0.681411,27.498679


# ***RISK MANAGER TESTS***

In [28]:
import cProfile
import pstats
import io
profiler = cProfile.Profile()
profiler.enable()
#run backtest



rm = RiskManager(evb_backtest.bars, None, None, '2023-01-01', '2023-12-31')
rm.OrderPicker.liquidity_threshold = 100
rm.OrderPicker.lookback = 10
order = rm.OrderPicker.get_order('TSLA', '2023-06-02', 'C', 2, 
                                  {'type': 'naked', 'specifics': [{'direction': 'long', 'rel_strike': 0.85, 'dte': 300, 'moneyness_width': 0.35}, {'direction': 'short', 'rel_strike': 0.6, 'dte': 300, 'moneyness_width': 0.35}], 'name': 'vertical_spread'})

profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

order

KeyError: 'signal_id'

In [15]:
from EventDriven.riskmanager import produce_order_candidates, populate_cache
import cProfile
import pstats
import io
profiler = cProfile.Profile()
profiler.enable()
#run backtest


candi = produce_order_candidates({'type': 'naked', 'specifics': [{'direction': 'long', 'rel_strike': 0.85, 'dte': 300, 'moneyness_width': 0.35}, {'direction': 'short', 'rel_strike': 0.6, 'dte': 300, 'moneyness_width': 0.35}], 'name': 'vertical_spread'},
                         'NVDA',
                         '2023-06-02',
                         'C')
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')


In [27]:
stats.print_stats()
print(stream.getvalue())

         3811024 function calls (3793904 primitive calls) in 41.362 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000   41.361    8.272 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        5    0.000    0.000   41.361    8.272 {built-in method builtins.exec}
      2/1    0.002    0.001   40.474   40.474 /Users/chiemelienwanisobi/cloned_repos/QuantTools/trade/helpers/decorators.py:37(wrapper)
        1    0.001    0.001   40.474   40.474 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/riskmanager.py:513(get_order)
     1795   27.676    0.015   27.676    0.015 {method 'acquire' of '_thread.lock' objects}
      524    0.004    0.000   27.638    0.053 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/threading.py:295(wait)
      396    0.004    0.000   25.366    0.064 /Users/chiemelienwanisobi

In [24]:
from EventDriven.riskmanager import produce_order_candidates, populate_cache
import cProfile
import pstats
import io
profiler = cProfile.Profile()
profiler.enable()
#run backtest
populate_cache(
    candi,
    evb_backtest.start_date,
    evb_backtest.end_date,
    evb_backtest.end_date,
)
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Updating Cache


In [25]:
stats.print_stats()
print(stream.getvalue())

KeyboardInterrupt: 

In [16]:
candi['long'][0].columns

Index(['Expiration', 'DTE', 'Strike', 'C', 'P', 'Spot', 'q', 'r',
       'relative_moneyness', 'moneyness_spread', 'dte_spread', 'ticker',
       'moneyness', 'TGT_DTE', 'right', 'option_id'],
      dtype='object', name='Right')

In [270]:
rm.option_data['TSLA20240315C180']

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-01-03,13.50,13.50,13.00,13.00,2,223,13.55,110,14.05,13.800,13.715165
2023-01-03,13.50,13.50,13.00,13.00,2,223,13.55,110,14.05,13.800,13.715165
2023-01-04,14.00,15.50,14.00,15.20,8,76,15.35,129,15.85,15.600,15.664634
2023-01-04,14.00,15.50,14.00,15.20,8,76,15.35,129,15.85,15.600,15.664634
2023-01-05,13.30,13.30,13.30,13.30,2,226,13.30,391,14.55,13.925,14.092139
2023-01-05,13.30,13.30,13.30,13.30,2,226,13.30,391,14.55,13.925,14.092139
2023-01-06,11.47,11.47,11.47,11.47,1,100,14.85,106,15.10,14.975,14.978641
2023-01-06,11.47,11.47,11.47,11.47,1,100,14.85,106,15.10,14.975,14.978641
2023-01-09,16.75,18.38,16.75,18.38,3,202,17.30,338,18.05,17.675,17.769444


In [253]:
RiskManager?

Init signature:
RiskManager(
    bars,
    events,
    initial_capital,
    start_date,
    end_date,
    portfolio_manager=None,
    price_on='close',
    option_price='Midpoint',
    sizing_type='delta',
    leverage=5.0,
)
Docstring:      <no docstring>
Init docstring:
initializes the RiskManager class

params:
bars: Bars: bars
events: Events: events
initial_capital: float: initial capital
File:           ~/cloned_repos/QuantTools/EventDriven/riskmanager.py
Type:           type
Subclasses:     

In [4]:
rm.OrderPicker.get_order?

Signature:
rm.OrderPicker.get_order(
    tick: str,
    date: str,
    right: str,
    max_close: str,
    order_settings: dict,
) -> dict
Docstring:
returns the order for the given tick, date, right, max_close, and order_settings

params:
tick: str: ticker to get the order for
date: str: date to get the order for
right: str: right of the option contract (P or C)
max_close: str: maximum close price
order_settings: dict: settings for the order
    example: {'type': 'naked',
                'specifics': [{'direction': 'long',
                'rel_strike': .900,
                'dte': 365,
                'moneyness_width': 0.15},
                {'direction': 'short',
                'rel_strike': .80,
                'dte': 365,
                'moneyness_width': 0.15}],

                'name': 'vertical_spread'}

returns:
dict: order
File:      ~/cloned_repos/QuantTools/EventDriven/riskmanager.py
Type:      method

In [2]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.assets.OptionStructure import OptionStructure
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import (change_to_last_busday, 
                                  is_USholiday, 
                                  is_busday, 
                                  setup_logger, 
                                  generate_option_tick, 
                                  get_option_specifics_from_key,
                                  identify_length,
                                  extract_numeric_value)
from scipy.stats import percentileofscore
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import (list_contracts, retrieve_openInterest, retrieve_eod_ohlc, retrieve_quote)
from pandas.tseries.offsets import BDay
from itertools import product
import pandas as pd
from trade.helpers.types import ResultsEnum
from copy import deepcopy
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathos.multiprocessing import ProcessingPool as Pool
import numpy as np
import time
chain_cache = {}
close_cache = {}
oi_cache = {}
spot_cache = {}


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

Using Proxy URL: http://18.232.166.224:5500/thetadata


## TEST IMPORT

In [7]:
from EventDriven.riskmanager import RiskManager, spot_cache, oi_cache, close_cache, chain_cache
from pandas.tseries.offsets import BDay
import pandas as pd

order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.15},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.15} 
            ],
            'name': 'vertical_spread'
        }
tick = 'AAPL'
date = '2024-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.get_order(tick, date, 'C', 5, order_settings)

{'L': ['AAPL20250620000235C'],
 'S': ['AAPL20250620000260C'],
 'trade_id': '&L:AAPL20250620000235C&S:AAPL20250620000260C',
 'close': 4.925000000000001}

## SAMPLE ORDER

In [3]:
tick = 'BAC'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.15},
                # {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.15} 
            ],
            'name': 'vertical_spread'
        }

## MULTIPROCESSING FUNCTION

In [4]:
from typing import List, Dict
from abc import ABC, abstractmethod
from pathos.multiprocessing import ProcessingPool as Pool
from pathos.multiprocessing import cpu_count
from pathos.pools import _ProcessPool
from threading import Thread
from functools import partial
from concurrent.futures import ThreadPoolExecutor

shutdown_event = False

def runProcesses(func, OrderedInputs: List[List], run_type: str = 'map') -> List:
    global shutdown_event
    try:

        pool = Pool(20)
        pool.restart()
        if run_type == 'map':
            results = pool.map(func, *OrderedInputs)
        elif run_type == 'amap':
            results = pool.amap(func, *OrderedInputs)
        elif run_type == 'uimap':
            results = pool.uimap(func, *OrderedInputs)
        elif run_type == 'imap':
            results = pool.imap(func, *OrderedInputs)

        else:
            raise ValueError(f'Run type {run_type} not recognized')

    except KeyboardInterrupt as e:

        shutdown_event = True
        shutdown(pool)
        raise

    except Exception as e:
        print('Error occured: ', e)
        shutdown(pool)


    finally:
        pool.close()
        pool.join()

    return results



def shutdown(pool):
    global shutdown_event
    shutdown_event
    shutdown_event = True
    pool.terminate()


## RELEVANT FUNCTIONS

In [5]:
def return_closePrice(id, date):
    global close_cache, spot_cache
    cache_key = f"{id}_{date}"
    close_data = close_cache[cache_key]
    close_data = close_data[~close_data.index.duplicated(keep = 'first')]
    close = close_data['Midpoint'][date]
    return close


def load_chain(date, ticker,  print_stderr = False):
        print(date, ticker) if print_stderr else None
        ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
        ## We will need to get two rows. 
        chain_key = f"{date}_{ticker}"
        with Context(end_date = date):
            if chain_key in chain_cache:
                Option_Chain = chain_cache[chain_key]
            else:
                start_time = time.time()
                Stock_obj = Stock(ticker, run_chain = False)
                end_time = time.time()
                print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                Option_Chain = Stock_obj.option_chain()
                Spot = Stock_obj.spot(ts = False)
                Spot = list(Spot.values())[0]
                Option_Chain['Spot'] = Spot
                Option_Chain['q'] = Stock_obj.div_yield()
                Option_Chain['r'] = Stock_obj.rf_rate
                chain_cache[chain_key] = Option_Chain



In [6]:
def populate_cache(order_candidates, date = '2024-03-12',):

    global close_cache, oi_cache, spot_cache

    tempholder1 = {}
    tempholder2 = {}

    ## Create necessary data structures
    ## Looping through the order candidates to get the necessary data, and organize into a list of lists that will be passed to runProcesses function
    for j, direction in enumerate(order_candidates):
        for i,data in enumerate(order_candidates[direction]):
            data[[ 'exp', 'strike', 'symbol']] = data[[ 'expiration', 'strike', 'ticker']]
            start = (pd.to_datetime(date) - BDay(20)).strftime('%Y-%m-%d')
            data[['end_date', 'start_date']] = date, start
            data['exp'] = data['exp'].dt.strftime('%Y-%m-%d')
            tempholder1[i+j] = (data[['symbol', 'end_date', 'exp', 'right', 'start_date', 'strike']].T.values.tolist())
            tempholder2[i+j] = data[['symbol', 'right', 'exp','strike']].T.values.tolist()

    ## Extending lists, to ensure only one runProcesses call is made, instead of run per side
    for i, data in tempholder1.items():
        if i == 0:
            OrderedList = data
            tickOrderedList = tempholder2[i]
        else:
            for position, vars in enumerate(data):
                OrderedList[position].extend(vars)
            for position, vars in enumerate(tempholder2[i]):
                tickOrderedList[position].extend(vars)

    
    eod_results = (runProcesses(retrieve_eod_ohlc, OrderedList, 'imap'))
    oi_results = (runProcesses(retrieve_openInterest, OrderedList, 'imap'))
    tick_results = (runProcesses(generate_option_tick, tickOrderedList, 'imap'))
    tick_results = list(set(tick_results))


    ## Save to Dictionary Cache
    for tick, eod, oi in zip(tick_results, eod_results, oi_results):
        cache_key = f"{tick}_{date}"
        close_cache[cache_key] = eod
        oi_cache[cache_key] = oi


    ## Test1: Run spot_cache process after close_cache has been populate.
    
    spot_results = list(runProcesses(return_closePrice, [tick_results, [date]*len(tick_results)], 'imap'))   
    for tick, spot in zip(tick_results, spot_results):
        cache_key = f"{tick}_{date}"
        spot_cache[cache_key] = spot


    ## Test2: We will edit the populate spot_cache populate function to make an api call instead of using the cache.



In [7]:
def produce_order_candidates(settings, tick, date, right = 'P'):
    order_candidates = {'long': [], 'short': []}
    for spec in settings['specifics']:
        chain = chain_details(date, tick, spec['dte'], spec['rel_strike'], right,  moneyness_width = spec['moneyness_width'])
        order_candidates[spec['direction']].append(chain)
    return order_candidates


def liquidity_check(id, date, pass_threshold = 250):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():

        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]

    start = (pd.to_datetime(date) - BDay(10)).strftime('%Y-%m-%d')
    oi_data = retrieve_openInterest(**transfer_dict, end_date=date, start_date=start)
    # print(f'Open Interest > {pass_threshold} for {id}:', oi_data.Open_interest.mean() )
    return oi_data.Open_interest.mean() > pass_threshold


def available_close_check(id, date, threshold = 0.7):
    cache_key = f"{id}_{date}"
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():
        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]
    
    if cache_key in close_cache:
        close_data_sample = close_cache[cache_key]
    else:
        start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
        close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
        close_cache[cache_key] = close_data_sample
    close_mask_series = close_data_sample.Close != 0
    return close_mask_series.sum()/len(close_mask_series) > threshold


def get_structure_price(tradeables, direction_index, date, tick, right = 'P'):
    pack_price = {}
    pack_dataframe = pd.DataFrame()
    pack_dataframe['close'] = 0

    for pack_i, pack in enumerate(tradeables):
        pack_close = 0
        for i, id in enumerate(pack):
            if id not in spot_cache:
                
                cache_key = f"{id}_{date}"
                sample_id = deepcopy(get_option_specifics_from_key(id))
                new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
                transfer_dict = {}
                for k, v in sample_id.items():
                    if k in new_dict_keys:
                        if k == 'strike':
                            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
                        else:
                            transfer_dict[new_dict_keys[k]] = sample_id[k]
                start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
                close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
                close_data_sample = close_data_sample[~close_data_sample.index.duplicated(keep = 'first')]
                close = close_data_sample['Midpoint'][date]
                spot_cache[cache_key] = close
            else:
                close = cache_key[id]
            pack_close += close * direction_index[i]
            pack_dataframe.at[pack_i, i] = id

        pack_dataframe.at[pack_i, 'close'] = pack_close
    return pack_dataframe




In [8]:
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15, print_stderr = False):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker) if print_stderr else None
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            chain_key = f"{date}_{ticker}"
            with Context(end_date = date):
                if chain_key in chain_cache:
                    Option_Chain = chain_cache[chain_key]
                else:
                    start_time = time.time()
                    Stock_obj = Stock(ticker, run_chain = False)
                    end_time = time.time()
                    print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                    Option_Chain = Stock_obj.option_chain()
                    Spot = Stock_obj.spot(ts = False)
                    Spot = list(Spot.values())[0]
                    Option_Chain['Spot'] = Spot
                    Option_Chain['q'] = Stock_obj.div_yield()
                    Option_Chain['r'] = Stock_obj.rf_rate
                    chain_cache[chain_key] = Option_Chain

                
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                
                
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Option_Chain_Filtered.Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                if float(moneyness_width) == 0.0:
                    option_details = Option_Chain_Filtered.sort_values('moneyness_spread', ascending=False).head(1)
                else:
                    option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & 
                                                        (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                
                if option_details.empty:
                    return None
                
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                option_details['right'] = right
                option_details.drop(columns = ['C','P'], inplace = True)
                option_details['option_id'] = option_details.apply(lambda x: generate_option_tick(symbol = x['ticker'], 
                                                                    exp = x['expiration'].strftime('%Y-%m-%d'), strike = float(x['strike']), right = x['right']), axis = 1)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise

        return return_dataframe.sort_values('relative_moneyness', ascending=False)
    else:
        return None, errors
    

# details= chain_details('2024-03-12', 'TSLA', 365, 0.7, moneyness_width = 0.00)

## CREATING ORDERPICKER AND TESTING

In [10]:
class OrderPicker:
    def __init__(self):
        self.liquidity_threshold = 250
        self.data_availability_threshold = 0.7
        self.lookback = 30

    def get_order(self, 
                  tick, 
                  date,
                  right, 
                  max_close,
                  order_settings):
        
        ## Create necessary data structures
        direction_index = {}
        str_direction_index = {}
        for indx, v in enumerate(order_settings['specifics']):
            if v['direction'] == 'long':
                str_direction_index[indx] = 'long'
                direction_index[indx] = 1
            elif v['direction'] == 'short':
                str_direction_index[indx] = 'short'
                direction_index[indx] = -1


        load_chain(date, 'TSLA')
        order_candidates = produce_order_candidates(order_settings, tick, date, right)

        if any([x2 is None for x in order_candidates.values() for x2 in x]):
            return {
                'result': ResultsEnum.MONEYNESS_TOO_TIGHT.value,
                'data': None
            } 


        populate_cache(order_candidates, date=date)


        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
                data = data[data.liquidity_check == True]
                data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
                order_candidates[direction][i] = data[data.available_close_check == True] 




        ## Filter Unique Combinations per leg.
        unique_ids = {'long': [], 'short': []}
        for direction in order_candidates:
            for i,data in enumerate(order_candidates[direction]):
                unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())

        ## Produce Tradeable Combinations
        tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
        tradeable_ids, unique_ids 

        ## Keep only unique combinations. Not repeating a contract.
        filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]

        ## Get the price of the structure
        ## Using List Comprehension to sum the prices of the structure per index
        results = [
            (*items, sum([direction_index[i] * spot_cache[f'{item}_{date}'] for i, item in enumerate(items)])) for items in filtered
        ]

        ## Convert to DataFrame, and sort by the price of the structure.
        return_dataframe = pd.DataFrame(results)
        cols = return_dataframe.columns.tolist()
        cols[-1] = 'close'
        return_dataframe.columns= cols
        return_dataframe = return_dataframe[(return_dataframe.close<= max_close) & (return_dataframe.close> 0)].sort_values('close', ascending = False).head(1)


        if return_dataframe.empty:
            return {
                'result': ResultsEnum.MONEYNESS_TOO_TIGHT.value,
                'data': None
            } 
            
        ## Rename the columns to the direction names
        return_dataframe.columns = list(str_direction_index.values()) + ['close']
        return_order = return_dataframe[list(str_direction_index.values())].to_dict(orient = 'list')
        return_order

        ## Create the trade_id with the direction and the id of the contract.
        id = ''
        for k, v in return_order.items():
            id += f"&{k[0].upper()}:{v[0]}"

        return_order['trade_id'] = id
        return_order['close'] = return_dataframe.close.values[0]
        return_dict = {
            'result': ResultsEnum.SUCCESSFUL.value,
            'data': return_order
        }


        return return_dict


order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.10},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.10} 
            ],
            'name': 'vertical_spread',
        }


tick = 'TSLA'
date = '2024-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'


picker = OrderPicker()
er = picker.get_order(tick, date, 'C', 10, order_settings)
er

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSLA20250620000215C'],
  'short': ['TSLA20250620000235C'],
  'trade_id': '&L:TSLA20250620000215C&S:TSLA20250620000235C',
  'close': 9.674999999999997}}

In [1]:
from multiprocessing import set_start_method
set_start_method("fork", force = True)
from trade.helpers.pools import runProcesses

def test_func(x):
    return x**2

if __name__ == '__main__':
    results = runProcesses(test_func, [[1,2,3,4,5]], 'imap')
list(results)

[1, 4, 9, 16, 25]

In [424]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        # self.symbol_list = self.bars.symbol_list
        self.OrderPicker = OrderPicker()


    def get_order(self, symbol, date, order_settings):
        pass



rm = RiskManager(None, None, 1000000)
rm.OrderPicker.get_order(tick, date, 'C', 10, order_settings)

{'L': ['TSLA20250321000165C'],
 'S': ['TSLA20250321000200C'],
 'trade_id': '&L:TSLA20250321000165C&S:TSLA20250321000200C',
 'close': 9.674999999999997}

## DRY RUN

In [35]:
max_close = 5


order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.01},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.01} 
            ],
            'name': 'vertical_spread'
        }




direction_index = {}
str_direction_index = {}
for indx, v in enumerate(order_settings['specifics']):
    if v['direction'] == 'long':
        str_direction_index[indx] = 'L'
        direction_index[indx] = 1
    elif v['direction'] == 'short':
        str_direction_index[indx] = 'S'
        direction_index[indx] = -1


load_chain(date, 'TSLA')
order_candidates = produce_order_candidates(order_settings, tick, date, right)

if any([x2 is None for x in order_candidates.values() for x2 in x]):
    return {
        'result': "MONEYNESS_TOO_TIGHT",
    } 


populate_cache(order_candidates, date=date)


for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data = data[data.liquidity_check == True]
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))
        order_candidates[direction][i] = data[data.available_close_check == True] 



## Filter Unique Combinations per leg.
unique_ids = {'long': [], 'short': []}
for direction in order_candidates:
    for i,data in enumerate(order_candidates[direction]):
        unique_ids[direction].append(data[(data.liquidity_check == True) & (data.available_close_check == True)].option_id.unique().tolist())

## Produce Tradeable Combinations
tradeable_ids = list(product(*unique_ids['long'], *unique_ids['short']))
tradeable_ids, unique_ids 

## Keep only unique combinations. Not repeating a contract.
filtered = [t for t in tradeable_ids if len(set(t)) == len(t)]

## Get the price of the structure
## Using List Comprehension to sum the prices of the structure per index
results = [
    (*items, sum([direction_index[i] * spot_cache[f'{item}_{date}'] for i, item in enumerate(items)])) for items in filtered
]

## Convert to DataFrame, and sort by the price of the structure.
return_dataframe = pd.DataFrame(results)
cols = return_dataframe.columns.tolist()
cols[-1] = 'close'
return_dataframe.columns= cols
return_dataframe = return_dataframe[(return_dataframe.close<= max_close) & (return_dataframe.close> 0)].sort_values('close', ascending = False).head(1)

## Rename the columns to the direction names
return_dataframe.columns = list(str_direction_index.values()) + ['close']
return_order = return_dataframe[list(str_direction_index.values())].to_dict(orient = 'list')
return_order

## Create the trade_id with the direction and the id of the contract.
id = ''
for k, v in return_order.items():
    id += f"&{k}:{v[0]}"

return_order['trade_id'] = id
return_order['close'] = return_dataframe.close.values[0]

return_order

TypeError: 'NoneType' object is not subscriptable

In [73]:
from trade.helpers.types import PositionData
def prineter(x,y) -> PositionData:
    return 

Type:        dict
String form: {}
Length:      0
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)

In [80]:
from typing import TypedDict
class PositionData( TypedDict): 
    long: list[str]
    short: list[str]

    def __setitem__(self, key, value):
        if (key == 'long' or key == 'short') and not isinstance(value, list):
            raise ValueError(f'{key} must be a list')
        
        if key == 'long':
            self.long = value
        elif key == 'short':
            self.short = value
        else:
            raise ValueError(f'Key {key} not recognized')
        super().__setitem__(key, value)

rr = PositionData()
rr['long']  = [3]
rr

{'long': [3]}

In [82]:
from typing import TypedDict

class Person(TypedDict):
    name: str
    age: int

# Static type checking
person: Person = {"name": "John", "age": 30}  # OK
person = {"name": "John"}  # Error (age is missing)

# Runtime behavior
print(person['name'])  # No runtime type enforcement, behaves like a regular dictionary


John


In [86]:
person: Person = {"name": "John", "age": 30}
person = {"name": "John"} 
person

{'name': 'John'}

In [18]:
return_dataframe

,L,S,close


In [92]:
import aiohttp
import asyncio
import time

async def fetch_data(session, url):
    print(f"Starting task: {url}")
    async with session.get(url) as response:
        data = await response.json()
        print(f"Completed task: {url}")
        return data

async def main():
    urls = [
        'https://api.github.com',
        'https://api.spacexdata.com/v4/launches/latest',
        'https://jsonplaceholder.typicode.com/todos/1'
    ]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        
        for result in results:
            print(result)

if __name__ == '__main__':
    start_time = time.time()
    asyncio.run(main())
    print(f"Total time taken: {time.time() - start_time} seconds")


Starting task: https://api.github.com
Starting task: https://api.spacexdata.com/v4/launches/latest
Starting task: https://jsonplaceholder.typicode.com/todos/1
Completed task: https://api.github.com
Completed task: https://api.spacexdata.com/v4/launches/latest
Completed task: https://jsonplaceholder.typicode.com/todos/1
{'current_user_url': 'https://api.github.com/user', 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}', 'authorizations_url': 'https://api.github.com/authorizations', 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}', 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}', 'emails_url': 'https://api.github.com/user/emails', 'emojis_url': 'https://api.github.com/emojis', 'events_url': 'https://api.github.com/events', 'feeds_url': 'https://api.github.com/feeds', 'followers_url': 'https://api.github.com/user/followers', 'following_url

Steps to producing an order:

- S1: RM recieves order settings from PM
- S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
- S3: RM assesses if option passes all checks
    - C1: Minimum Available close
    - C2: Liquidity (Open Interest)
    - C2.5: (for Spreads only) Ensure both legs are not the same
    - Optional, to extend:
    - C3: Bid-Ask Spread
    
- S4: Return picked order to portfolio manager, which places the order. 
- Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [2]:
import os
eval(os.environ['ASYNC'])

False